In [1]:
from quantumnet.components import Controller, Network
from quantumnet.utils import generate_random_request, register_request
from quantumnet.objects import Request, time
import asyncio
path = "resultados/proativo/sync.csv"
rede = Network()
controlador = Controller(rede)
row, col = 3, 4
rede.set_ready_topology("Grade", row, col)
# Criação de requisições
fmin_range = (0.5, 1)
neprs_range = (2, 4)
num_requests = 1000
requests = list(generate_random_request(len(rede.hosts)-1, fmin_range, neprs_range) for i in range(num_requests))

Hosts inicializados
Canais inicializados
Pares EPRs adicionados


In [2]:
hosts = rede.hosts
for alice in hosts:
    print(f"Adicionando regras para {alice}")
    for bob in hosts:
        controlador.add_match_route_rule_in_host_proactive(alice, bob, 0.5, 5)
        controlador.add_match_route_rule_in_host_proactive(alice, bob, 0.9, 5)
    rede.get_host(alice).draw_flow_table()
    time.increment()

Adicionando regras para 0
+--------------+----------------------+--------------------+
| Match        | Route                | Rule               |
+==============+======================+====================+
| (0, 0.5, 5)  | [0]                  | BasicRuleProactive |
+--------------+----------------------+--------------------+
| (0, 0.9, 5)  | [0]                  | BasicRuleProactive |
+--------------+----------------------+--------------------+
| (1, 0.5, 5)  | [0, 1]               | BasicRuleProactive |
+--------------+----------------------+--------------------+
| (1, 0.9, 5)  | [0, 1]               | BasicRuleProactive |
+--------------+----------------------+--------------------+
| (2, 0.5, 5)  | [0, 1, 2]            | BasicRuleProactive |
+--------------+----------------------+--------------------+
| (2, 0.9, 5)  | [0, 1, 2]            | BasicRuleProactive |
+--------------+----------------------+--------------------+
| (3, 0.5, 5)  | [0, 1, 2, 3]         | BasicRuleProactive 

In [3]:
memoria = rede.get_host(1).memory
qubit = memoria[0]
print(qubit.get_initial_fidelity())
print(qubit.get_current_fidelity())
print(time.get_current_time())

1
0.8869204367171575
12


In [4]:
def simulate_requests_with_timeslots(rede, requests, controlador):
    """
    Simula o processamento de requests com a introdução de timeslots.
    - Demora 1 time-slot se a regra existe.
    - Demora 3 time-slots se uma nova regra precisa ser adicionada.
    """
    current_time = 0  # Tempo global da simulação
    for request in requests:
        print(f"[Time {current_time}]")
        print(f"Request: {request}, Alice: {request.alice}, Bob: {request.bob}, Fmin: {request.fmin}, Neprs: {request.neprs}")
        alice = rede.get_host(request.alice)
        rule = alice.find_rule_by_request(request)

        if rule == False:  # Caso não exista um match na tabela
            request.endtime = current_time
            # Registra no CSV como um novo registro
            register_request(request, "Descartado", path)
        else:  # Caso já exista a regra
            request.starttime = current_time
            controlador.run_rule(rule[1])
            request.endtime = current_time
            
            # Registra no CSV como já registrado
            register_request(request, "Já Registrado", path)
        
        current_time += 1  # Delay de 1 time-slot para execução
        # Exibir informações da requisição
        print(f"Request {request}: Start Time = {request.starttime}, End Time = {request.endtime}")

simulate_requests_with_timeslots(rede, requests, controlador)

[Time 0]
Request: R-0927E0, Alice: 6, Bob: 1, Fmin: 0.75, Neprs: 4
Passo: 1
Executando ação: CreateEPR(6-5)
Executando ação: CreateEPR(5-1)
Passo: 2
Executando ação: Swap(6-5-1)
Running Swap(6-5-1)
Request R-0927E0: Start Time = 0, End Time = 0
[Time 1]
Request: R-F8A0C0, Alice: 7, Bob: 4, Fmin: 0.63, Neprs: 3
Passo: 1
Executando ação: CreateEPR(7-6)
Executando ação: CreateEPR(6-5)
Executando ação: CreateEPR(5-4)
Passo: 2
Executando ação: Swap(7-6-5)
Running Swap(7-6-5)
Request R-F8A0C0: Start Time = 1, End Time = 1
[Time 2]
Request: R-1097F0, Alice: 5, Bob: 2, Fmin: 0.73, Neprs: 4
Passo: 1
Executando ação: CreateEPR(5-1)
Executando ação: CreateEPR(1-2)
Passo: 2
Executando ação: Swap(5-1-2)
Running Swap(5-1-2)
Request R-1097F0: Start Time = 2, End Time = 2
[Time 3]
Request: R-A07170, Alice: 8, Bob: 2, Fmin: 0.79, Neprs: 3
Passo: 1
Executando ação: CreateEPR(8-4)
Executando ação: CreateEPR(4-0)
Executando ação: CreateEPR(0-1)
Executando ação: CreateEPR(1-2)
Passo: 2
Executando ação: Swa